In [2]:
import pandas as pd


In [3]:
hero_network_df = pd.read_csv("./hero-network.csv")

hero_network_df.head()

,hero1,hero2
0,"LITTLE, ABNER",PRINCESS ZANDA
1,"LITTLE, ABNER",BLACK PANTHER/T'CHAL
2,BLACK PANTHER/T'CHAL,PRINCESS ZANDA
3,"LITTLE, ABNER",PRINCESS ZANDA
4,"LITTLE, ABNER",BLACK PANTHER/T'CHAL


In [4]:
# strip spaces from hero names
hero_network_df["hero1"] = hero_network_df["hero1"].apply(lambda x: x.strip())
hero_network_df["hero2"] = hero_network_df["hero2"].apply(lambda x: x.strip())

hero_network_df.head()

,hero1,hero2
0,"LITTLE, ABNER",PRINCESS ZANDA
1,"LITTLE, ABNER",BLACK PANTHER/T'CHAL
2,BLACK PANTHER/T'CHAL,PRINCESS ZANDA
3,"LITTLE, ABNER",PRINCESS ZANDA
4,"LITTLE, ABNER",BLACK PANTHER/T'CHAL


In [5]:
# all_heroes contains each hero name once
all_heroes = set()

for row in hero_network_df.index:
    all_heroes.add(hero_network_df["hero1"][row])
    all_heroes.add(hero_network_df["hero2"][row])
    
print(len(all_heroes))
# all_heroes

6421


In [6]:
from collections import defaultdict

# create an undirected graph (adjacency list) for use with BFS
# this does not show edge weights (no count of edges between characters)
undir_hero_map = defaultdict(set)

for row in hero_network_df.index:
    hero1 = hero_network_df["hero1"][row]
    hero2 = hero_network_df["hero2"][row]
    
    undir_hero_map[hero1].add(hero2)
    undir_hero_map[hero2].add(hero1)
    
print(len(undir_hero_map.keys()))
# undir_hero_map

6421


In [7]:
# get the number of edges/links (not weighted)
# from undirected graph
total_num_edges = 0

for hero in undir_hero_map.keys():
    edge_length = len(undir_hero_map[hero])
    total_num_edges += edge_length
    
total_num_edges /= 2

total_num_edges

167106.0

In [8]:
# export basic csv with hero name to edge count (first-degree relations)

ordered_heroes = list(all_heroes)

first_deg_df = pd.DataFrame(data={"hero":[hero for hero in ordered_heroes], "count":[len(undir_hero_map[hero]) for hero in ordered_heroes]})

first_deg_df.to_csv("./first_degree.csv", index=False)

first_deg_df = first_deg_df.sort_values(by='count', ascending=False)

#top 60 heroes by first-degree connections - most of these have appeared in movies already!
first_deg_df.head(60)

,hero,count
4589,CAPTAIN AMERICA,1904
2088,SPIDER-MAN/PETER PAR,1737
5447,IRON MAN/TONY STARK,1521
1592,THING/BENJAMIN J. GR,1416
4954,MR. FANTASTIC/REED R,1377
2987,WOLVERINE/LOGAN,1368
1739,HUMAN TORCH/JOHNNY S,1361
587,SCARLET WITCH/WANDA,1322
3901,THOR/DR. DONALD BLAK,1289
5999,BEAST/HENRY &HANK& P,1265


In [9]:
from collections import deque

# basic connectivity test - can do now that we have 
# the undirected adjacency list 

# test how many groups of heroes there are (connectivity)

# helper function that gives all heroes connected to a given hero
def basicBFS(hero, graph_map):
    queue = deque([hero])
    seen = set([hero])
    
    while(len(queue) > 0):
        curr_hero = queue.popleft()
        
        # add all first-degree heroes not in seen
        for adjacent_hero in graph_map[curr_hero]:
            if(adjacent_hero not in seen):
                queue.append(adjacent_hero)
                seen.add(adjacent_hero)
            
    
    return seen

# connectivity function - returns the number of heroes in each group and number of unconnected groups
def connectivity(hero_set, graph_map):
    all_groups = []
    all_seen = set()
    count = 0
    
    for hero in graph_map.keys():
        count += 1
        if(hero not in all_seen):
            hero_group = basicBFS(hero, graph_map)
            all_groups.append(len(hero_group))
            for connected_hero in hero_group:
                all_seen.add(connected_hero)
                
    print("Number of Groupings and Hero Count:", all_groups)
    print("Total Number of Heroes Seen (should match total hero count):", count)
                
    return all_groups

print(connectivity(all_heroes, undir_hero_map))

Number of Groupings and Hero Count: [6403, 9, 7, 2]
Total Number of Heroes Seen (should match total hero count): 6421
[6403, 9, 7, 2]


In [10]:
# basic BFS for getting hero degree of separation
# includes information about the links between the heroes specified

def hero_BFS(hero1, hero2, graph_map):    
    queue = deque()
    queue.append((hero1, [hero1]))
    seen = set([hero1])
    
    while(len(queue) > 0):
        curr_hero, hero_chain = queue.popleft()
        
        # if curr_hero is hero2, end loop
        if(curr_hero == hero2):
            return hero_chain
        
        # otherwise, add all unseen heroes to queue, with chain
        for new_hero in graph_map[curr_hero]:
            if(new_hero not in seen):
                new_hero_chain = hero_chain.copy()
                new_hero_chain.append(new_hero)
                
                queue.append((new_hero, new_hero_chain))
                
                seen.add(new_hero)
#     print(seen)
    return ["Not connected!"]
            
# test
hero_BFS('IRON MAN/TONY STARK', "EMPRESS S'BYLL [SKRU", undir_hero_map)

['IRON MAN/TONY STARK', 'NEBULA', "EMPRESS S'BYLL [SKRU"]

In [26]:
# basic BFS for getting all routes to hero at min. degree of separation
# includes information about the links between the heroes specified
# as an array of all possible paths

def all_paths_BFS(hero1, hero2, graph_map):    
    # stop at the minimum degree, and return all paths
    # that have hero2 as the last value
    min_degree = len(hero_BFS(hero1, hero2, graph_map))
    all_paths = []
    
    queue = deque()
    queue.append((hero1, [hero1]))
    seen = set([hero1])
    
    while(len(queue[0][1]) <= min_degree):
        curr_hero, hero_chain = queue.popleft()
        
        # if curr_hero is hero2, end loop
        if(curr_hero == hero2):
            all_paths.append(hero_chain)
        
        # otherwise, add all unseen heroes to queue, with chain
        for new_hero in graph_map[curr_hero]:
            if(new_hero not in seen or new_hero == hero2):
                new_hero_chain = hero_chain.copy()
                new_hero_chain.append(new_hero)
                
                queue.append((new_hero, new_hero_chain))
                
                seen.add(new_hero)
#     print(seen)
    return all_paths
            
# test
all_paths_BFS('LITTLE, ABNER', "FAITH", undir_hero_map)

[['LITTLE, ABNER', 'IRON MAN IV/JAMES R.', 'MAKKARI/MIKE KHARY/I', 'FAITH'],
 ['LITTLE, ABNER', 'IRON MAN IV/JAMES R.', 'MOONDRAGON/HEATHER D', 'FAITH'],
 ['LITTLE, ABNER', 'JOCASTA', 'QUASAR III/WENDELL V', 'FAITH'],
 ['LITTLE, ABNER', 'JOCASTA', 'LORD CHAOS', 'FAITH'],
 ['LITTLE, ABNER', 'JOCASTA', 'MASTER ORDER', 'FAITH'],
 ['LITTLE, ABNER', "BLACK PANTHER/T'CHAL", 'IN-BETWEENER', 'FAITH'],
 ['LITTLE, ABNER', "BLACK PANTHER/T'CHAL", "KISMET/J'RIDIA STARD", 'FAITH'],
 ['LITTLE, ABNER', "BLACK PANTHER/T'CHAL", 'THANOS', 'FAITH'],
 ['LITTLE, ABNER', "BLACK PANTHER/T'CHAL", 'SILVER SURFER/NORRIN', 'FAITH'],
 ['LITTLE, ABNER', "BLACK PANTHER/T'CHAL", 'MEPHISTO', 'FAITH'],
 ['LITTLE, ABNER', 'BINARY/CAROL DANVERS', 'BALLANTINE, KAYLA', 'FAITH'],
 ['LITTLE, ABNER', 'THOR/DR. DONALD BLAK', 'LIVING TRIBUNAL', 'FAITH'],
 ['LITTLE, ABNER', 'IRON MAN/TONY STARK', 'TANAKA, KENJIRO', 'FAITH']]

In [28]:
import json

# output JSON version of all_paths output that is compatible with 
# a D3 sankey visualization -> https://www.d3-graph-gallery.com/graph/sankey_basic.html

def hero_paths_to_json(all_paths_arr):
    # degree of separation
    sep_degree = len(all_paths_arr[0])
    
    # all heroes present in the sankey, no duplicates
    heroes_seen = set([b for a in all_paths_arr for b in a])
    
    # index values are the node numbers in the json
    heroes_ordered = list(heroes_seen)
    
    # used for mapping hero names to indices - for output json
    hero_index_map = {}
    for i, hero in enumerate(heroes_ordered):
        hero_index_map[hero] = i
        
    # create nodes list of "objects" - first part of output json
    nodes = [{"name": hero, "node": i} for i, hero in enumerate(heroes_ordered)]
        
    # tuple dictionary for weighting flows/edges
    edge_dict = defaultdict(int)
    
    for arr in all_paths_arr:
        for i in range(sep_degree-1):
            from_hero = arr[i]
            to_hero = arr[i+1]
            edge_dict[(from_hero, to_hero)] += 1
        
    links = [{"source":hero_index_map[key[0]], "target":hero_index_map[key[1]], "value":value} for key, value in edge_dict.items()]
    
    output_to_json = {"nodes":nodes, "links":links}
    
    return output_to_json
    
test_sankey_json = hero_paths_to_json(all_paths_BFS('LITTLE, ABNER', "FAITH", undir_hero_map))

with open('sankey_data.json', 'w') as outfile:
    json.dump(test_sankey_json, outfile)

# json.dumps(test_sankey_json)

In [12]:
# max degrees of separation for a character

def maxSeparation(hero, graph_map):
    queue = deque()
    queue.append((hero, 0))
    seen = set([hero])
    
    while(len(queue) > 0):
        curr_hero, curr_distance = queue.popleft()
        
        # add all first-degree heroes not in seen
        for adjacent_hero in graph_map[curr_hero]:
            if(adjacent_hero not in seen):
                queue.append((adjacent_hero, curr_distance+1))
                seen.add(adjacent_hero)                
    
    return curr_distance

# test
maxSeparation("FAITH", undir_hero_map)

4

In [13]:
# # calculate the maximum distance in the graph (width) - 
# # by calculating the max number of degrees of separation for each character
# # takes some time, so usually commented out after first file creation

# # NOTE: the separate groups are not split up here - could look into that in the future!

# hero_dist_map = defaultdict(int)

# for hero in all_heroes:
#     max_dist = maxSeparation(hero, undir_hero_map)
#     hero_dist_map[hero] = max_dist
    
# # hero_dist_map

In [14]:
# # max degree of separation for Marvel characters
# print("The maximum degree of separation in this network is:", max(hero_dist_map.values()))
# print("The smallest maximum degree of separation in this network is:", min(hero_dist_map.values()))
# print("The average maximum degree of separation in this network is:", sum(hero_dist_map[hero] for hero in all_heroes)/len(all_heroes))

# # save graph distances to a file

# max_dist_df = pd.DataFrame(data={"hero":ordered_heroes, "max_dist":[hero_dist_map[hero] for hero in ordered_heroes]})

# max_dist_df.to_csv("./max_distances.csv", index=False)

# max_dist_df.head()

In [15]:
# import max_dist_df if already created

max_dist_df = pd.read_csv("./max_distances.csv")

max_dist_df.head()

,hero,max_dist
0,"MCLAREN, MEGAN",4
1,SPANKER/FRED HOVEL,4
2,SERAPH,4
3,ULTRA-MAX,4
4,VIRTUAL REALITY,4


In [16]:
# heroes not part of the main network comprise the minimum max distances
# - 3 disconnected networksfrom main network!

max_dist_df.loc[max_dist_df["max_dist"] == 1]

,hero,max_dist
1146,STEEL SPIDER/OLLIE O,1
1503,"DARLEGUNG, GEN.",1
2051,"ASHER, MICHAEL",1
2283,SWORDSMAN IV/,1
2287,MANT/ERNEST,1
2465,STERLING,1
2767,OSWALD,1
3124,AMAZO-MAXI-WOMAN/,1
3624,MASTER OF VENGEANCE,1
3660,HOFFMAN,1


In [17]:
print(hero_BFS("AMAZO-MAXI-WOMAN/", "FAITH", undir_hero_map))

['Not connected!']


In [18]:
# output all the heroes to a json file

max_dist_df["hero"].to_json(path_or_buf='./names.json', orient="values")